### Comunicating with google api and generating BLEU score

In [1]:
text_file1 = open("/home/arthurtelles/Downloads/port_test.txt", "r+").read()
len(text_file1)

555845

In [2]:
type(text_file1)

str

In [3]:
text_file1[:100]

'eu não gosto da minha sala .\nnão funcionará .\nos pais do tom moram em uma casa grande .\nisso é mais '

In [4]:
text_list = text_file1.split('\n')
len(text_list)

16513

In [5]:
text_list[:5]

['eu não gosto da minha sala .',
 'não funcionará .',
 'os pais do tom moram em uma casa grande .',
 'isso é mais difícil do que eu esperava .',
 'o francês do tom é muito bom .']

In [6]:
"""Translates text into the target language.

Target must be an ISO 639-1 language code.
See https://g.co/cloud/translate/v2/translate-reference#supported_languages
"""
from google.cloud import translate_v2 as translate
import six
from tqdm import tqdm

translate_client = translate.Client()

# RODAR export GOOGLE_APPLICATION_CREDENTIALS="/home/arthurtelles/gtranslate-api-290022-4899f0c9d3f7.json"

# text = 'ola eu gosto de framboesa'
result_list = []

# Text can also be a sequence of strings, in which case this method
# will return a sequence of results for each text.
for line in tqdm(text_list):
    if isinstance(line, six.binary_type):
        line = line.decode("utf-8")
    result = translate_client.translate(line, target_language='en')
    result_list.append(result)

# print(u"Text: {}".format(result["input"]))
# print(u"Translation: {}".format(result["translatedText"]))
# print(u"Detected source language: {}".format(result["detectedSourceLanguage"]))

100%|██████████| 16513/16513 [1:18:09<00:00,  3.52it/s]


In [10]:
text_file2 = open("eng_gtranslate.txt", "a")
for line in result_list:
    text_file2.write(f"{line['translatedText']}\n")
text_file2.close()

In [20]:
result_list[0]['translatedText'].encode('utf-8').decode('utf-8')

'i don&#39;t like my room.'

In [23]:
import pandas as pd

for i, line in enumerate(result_list):
    new_line = line
    new_line['translatedText'] = new_line['translatedText'].replace('&#39;', "'")
    result_list[i] = new_line
pd.DataFrame(result_list)

,translatedText,detectedSourceLanguage,input
0,i don't like my room.,pt,eu não gosto da minha sala .
1,it won't work.,pt,não funcionará .
2,tom's parents live in a big house.,pt,os pais do tom moram em uma casa grande .
3,this is more difficult than I expected.,pt,isso é mais difícil do que eu esperava .
4,the tone French is very good.,pt,o francês do tom é muito bom .
...,...,...,...
16508,tone was not clear.,pt,tom não foi claro .
16509,tom was really sad.,pt,tom estava realmente triste .
16510,Tom will not help Mary.,pt,tom não ajudará a mary .
16511,watch the front.,pt,vigia a frente .


,translatedText,detectedSourceLanguage,input
16493,I can work here ?,pt,posso trabalhar aqui ?
16494,this information is not correct.,pt,essa informação não é correta .
16495,they will come back.,pt,eles voltarão .
16496,let me sit in the back.,pt,deixe me sentar atrás .
16497,who conducts the tests?,pt,quem conduz os testes ?
16498,Please pack the letters carefully.,es,por favor embaralhe as cartas cuidadosamente .
16499,where did tom live?,pt,onde tom morou ?
16500,welcome back . we miss you .,pt,bem vindo de volta . sentimos a sua falta .
16501,the cats are on the roof.,pt,os gatos estão no telhado .
16502,I saw a man's face in the window.,pt,eu vi o rosto de um homem na janela .


**Loading references**

In [40]:
ref_file = open("/home/arthurtelles/Downloads/eng_test.txt", "r+").read()
ref_list = ref_file.split('\n')

In [28]:
print(len(ref_list), len(result_list))

16513 16513


In [36]:
from nltk.translate.bleu_score import corpus_bleu
from typing import List, Tuple, Dict, Set, Union

def compute_corpus_level_bleu_scoreresult_list(references: List[List[str]], hypotheses: List[str]) -> float:
    """ Given decoding results and reference sentences, compute corpus-level BLEU score.
    @param references (List[List[str]]): a list of gold-standard reference target sentences
    @param hypotheses (List[Hypothesis]): a list of hypotheses, one for each reference
    @returns bleu_score: corpus-level BLEU score
    """
    if references[0][0] == '<s>':
        references = [ref[1:-1] for ref in references]
    bleu_score = corpus_bleu([[ref] for ref in references],
                             [hyp for hyp in hypotheses])
    return bleu_score

translated_list = [result['translatedText'] for result in result_list]
compute_corpus_level_bleu_score(list(ref_list), translated_list)

0.7647256496733583

**Loading transformer translations**

In [39]:
nmt_file = open("/home/arthurtelles/Downloads/translations.txt", "r+").read()
nmt_list = nmt_file.split('\n')

In [41]:
print(len(nmt_list), len(result_list))

16513 16513


In [42]:
compute_corpus_level_bleu_score(list(ref_list), nmt_list)

0.6834019971487161

### Counting unique words in vocabulary

In [12]:
133742/165115

0.8099930351573146

In [10]:
text_file = open("/home/arthurtelles/Documents/other_repositories/pt_customized_transformer/data/port.txt", "r+").read()
text_file_test = open("/home/arthurtelles/Documents/other_repositories/pt_customized_transformer/data/port_test.txt", "r+").read()
text_file_train = open("/home/arthurtelles/Documents/other_repositories/pt_customized_transformer/data/port_train.txt", "r+").read()
print(len(text_file.split('\n')), len(text_file_test.split('\n')), len(text_file_train.split('\n')))

165115 16513 133742


In [2]:
text_file[:500]

'vai .\nvá .\nir .\noi .\ncorre !\ncorra !\ncorram !\ncorre !\ncorra !\ncorram !\nquem ?\nque\nuau !\nnossa !\nwow !\nfogo !\nincêndio !\nchama !\najuda !\nsocorro !\npule !\npula !\npulem .\npule .\npulam .\npare !\nparem !\nparada !\nponto !\npará !\nespere !\naguarde !\nespere !\nesperem .\nsiga em frente .\nvá !\ncontinue .\nsiga adiante .\noi .\nalô .\nolá !\nalô !\neu corri .\neu sei .\neu entendo .\nestou vendo .\neu vejo .\neu assisto .\neu tento .\ntento .\nganhei !\neu venci .\nah não !\nrelaxe !\nrelaxa !\ntiro !\nsorria .\nsorriam .\natacar '

In [3]:
import string
string.punctuation

'!"#$%&\'()*+,-./:;<=>?@[\\]^_`{|}~'

In [13]:
import re, string

def generate_corpus_statistics(text_file: str):
    ftext_file = text_file.replace('\n', '')
    for punct in string.punctuation:
        ftext_file = ftext_file.replace(punct, '')
    ftext_list = ftext_file.split(' ')
    print(f'Text contains {len(ftext_list)} words and {len(set(ftext_list))} unique words')
generate_corpus_statistics(text_file)
generate_corpus_statistics(text_file_test)
generate_corpus_statistics(text_file_train)

Text contains 992970 words and 20880 unique words
Text contains 99713 words and 7768 unique words
Text contains 803877 words and 19372 unique words


In [14]:
19372/20880

0.9277777777777778